In [1]:
from sqlalchemy import create_engine
from db_constants import Base
from sqlalchemy import func
import pandas as pd
from models import ParentPair, ChildPair
from sqlalchemy.orm import sessionmaker
from sqlalchemy import String
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_15379/249799025.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
# print(len(qry.all()))
# print(qry.all())

# # Convert the result to a pandas DataFrame
# data = {'id': [1, 2],
#         'symbol': ['ETH-BTC-USDT', 'ETH-BTC-TUSD'],
#         'sub_pairs_list': ['BTCUSDT, ETHBTC, ETHUSDT', 'BTCTUSD, ETHBTC'],
#         'best_bid_prices_list': ['43008.88, 0.05355, 2303.76', '43558.59, 0.05355'],
#         'best_ask_prices_list': ['43008.89, 0.05356, 2303.77', '43558.6, 0.05356']}


result = DataLoader().get_latest_data()
df = pd.DataFrame(result, columns=['id', 'assets', 'sub_pairs_list', 'best_bid_prices_list', 'best_ask_prices_list'])
df[['sub_pair__B/C', 'sub_pair__A/B', 'sub_pair__A/C']] = df['sub_pairs_list'].str.split(', ', expand=True)
df[['bid_price__B/C', 'bid_price__A/B', 'bid_price__A/C']] = df['best_bid_prices_list'].str.split(', ', expand=True).astype(float)

# Split 'best_ask_prices_list' into three columns and convert values to float
df[['ask_price_A', 'ask_price_B', 'ask_price_C']] = df['best_ask_prices_list'].str.split(', ', expand=True).astype(float)
# Display the DataFrame
columns_to_check = ['sub_pair__B/C', 'sub_pair__A/B', 'sub_pair__A/C', 'bid_price__B/C', 'bid_price__A/B', 'bid_price__A/C',
                     'ask_price_A', 'ask_price_B', 'ask_price_C']

# Drop rows where any of the specified columns contain NaN
df = df.dropna(subset=columns_to_check)
pd.set_option('display.max_rows', None)
print(len(df))
display(df)


8


,id,assets,sub_pairs_list,best_bid_prices_list,best_ask_prices_list,sub_pair__B/C,sub_pair__A/B,sub_pair__A/C,bid_price__B/C,bid_price__A/B,bid_price_A/C
0,1,ETH-BTC-USDT,"BTCUSDT, ETHBTC, ETHUSDT","43032.33, 0.0535, 2301.8","43032.34, 0.05351, 2301.81",4.303234e+04,ETHBTC,ETHUSDT,4.303233e+04,5.351000e-02,2301.81000
1,2,ETH-BTC-TUSD,"BTCTUSD, ETHBTC, ETHTUSD","43577.14, 0.0535, 2330.82","43577.15, 0.05351, 2331.29",4.357715e+04,ETHBTC,ETHTUSD,4.357714e+04,5.351000e-02,2331.29000
3,4,ETH-BTC-USDC,"BTCUSDC, ETHBTC, ETHUSDC","43011.55, 0.0535, 2300.94","43011.56, 0.05351, 2300.95",4.301156e+04,ETHBTC,ETHUSDC,4.301155e+04,5.351000e-02,2300.95000
6,7,ETH-BTC-TRY,"BTCTRY, ETHBTC, ETHTRY","1324136.0, 0.0535, 70833.0","1324646.0, 0.05351, 70853.0",1.324646e+06,ETHBTC,ETHTRY,1.324136e+06,5.351000e-02,70853.00000
7,8,ETH-BTC-EUR,"BTCEUR, ETHBTC, ETHEUR","39899.13, 0.0535, 2134.09","39900.65, 0.05351, 2134.1",3.990065e+04,ETHBTC,ETHEUR,3.989913e+04,5.351000e-02,2134.10000
8,9,ETH-BTC-ZAR,"BTCZAR, ETHBTC, ETHZAR","830509.0, 0.0535, 44417.0","831152.0, 0.05351, 44474.0",8.311520e+05,ETHBTC,ETHZAR,8.305090e+05,5.351000e-02,44474.00000
11,12,ETH-BTC-BIDR,"BTCBIDR, ETHBIDR, ETHBTC","675409065.0, 36001408.0, 0.0535","676378717.0, 36185436.0, 0.05351",6.763787e+08,ETHBIDR,ETHBTC,6.754091e+08,3.618544e+07,0.05351
13,14,ETH-BTC-DAI,"BTCDAI, ETHBTC, ETHDAI","43007.25, 0.0535, 2300.85","43028.32, 0.05351, 2300.86",4.302832e+04,ETHBTC,ETHDAI,4.300725e+04,5.351000e-02,2300.86000
